<a href="https://colab.research.google.com/github/endteamschoolofai/END/blob/main/Session9/Assignment_9_Learning_Phrase_Representations_using_RNN_Encoder_Decoder_for_Statistical_Machine_Translation_dataset_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

In this second notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078). This model will achieve improved test perplexity whilst only using a single layer RNN in both the encoder and the decoder.

## Introduction

Let's remind ourselves of the general encoder-decoder model.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

We use our encoder (green) over the embedded source sequence (yellow) to create a context vector (red). We then use that context vector with the decoder (blue) and a linear layer (purple) to generate the target sentence.

In the previous model, we used an multi-layered LSTM as the encoder and decoder.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq4.png?raw=1)

One downside of the previous model is that the decoder is trying to cram lots of information into the hidden states. Whilst decoding, the hidden state will need to contain information about the whole of the source sequence, as well as all of the tokens have been decoded so far. By alleviating some of this information compression, we can create a better model!

We'll also be using a GRU (Gated Recurrent Unit) instead of an LSTM (Long Short-Term Memory). Why? Mainly because that's what they did in the paper (this paper also introduced GRUs) and also because we used LSTMs last time. To understand how GRUs (and LSTMs) differ from standard RNNS, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) link. Is a GRU better than an LSTM? [Research](https://arxiv.org/abs/1412.3555) has shown they're pretty much the same, and both are better than standard RNNs. 

## Preparing Data

All of the data preparation will be (almost) the same as last time, so we'll very briefly detail what each code block does. See the previous notebook for a recap.

We'll import PyTorch, TorchText, spaCy and a few standard modules.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext import data 
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
import pandas as pd
import math

Then set a random seed for deterministic results/reproducability.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset 4 - Customer Support on Twitter:

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip /content/drive/MyDrive/end/archive.zip

Archive:  /content/drive/MyDrive/end/archive.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


In [6]:
train_df_fourth = pd.read_csv('twcs/twcs.csv')
train_df_fourth.head(10)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
7,11,sprintcare,False,Tue Oct 31 22:10:35 +0000 2017,@115713 This is saddening to hear. Please shoo...,NaN,12.0
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0


In [7]:
len(train_df_fourth)

2811774

In [8]:
df = train_df_fourth.dropna() #math.isnan(train_df_fourth.response_tweet_id[1])
df.head(10)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0
10,16,115713,True,Tue Oct 31 20:00:43 +0000 2017,@sprintcare Since I signed up with you....Sinc...,15,17.0
11,17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,16,18.0
15,21,Ask_Spectrum,False,Tue Oct 31 22:14:37 +0000 2017,@115716 What information is incorrect? ^JK,"22,23",24.0


In [9]:
len(df)

976810

In [10]:
df = df.set_index("tweet_id")
df.head(15)

,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
tweet_id,,,,,,
1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0
15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0
16,115713,True,Tue Oct 31 20:00:43 +0000 2017,@sprintcare Since I signed up with you....Sinc...,15,17.0
17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,16,18.0


In [11]:
df = df.head(5000)
len(df)

5000

In [12]:
questions = []
answers = []
for i in df.index:
  responses = df.response_tweet_id[i].split(',')
  responses = [int(j) for j in responses]
  for res in responses:
    if (res in df.index):
      ques = df.text[i]
      #print(ques)
      ans = df.text[res]
      #print(ans)
      questions.append(ques)
      answers.append(ans)
    

In [13]:
len(questions), len(answers)

(3376, 3376)

In [14]:
questions[:15], answers[:15]

(['@sprintcare I have sent several private messages and no one is responding as usual',
  '@115712 Please send us a Private Message so that we can further assist you. Just click ‘Message’ at the top of your profile.',
  '@sprintcare I did.',
  '@115712 Can you please send us a private message, so that I can gain further details about your account?',
  "@115713 We understand your concerns and we'd like for you to please send us a Direct Message, so that we can further assist you. -AA",
  '@sprintcare Since I signed up with you....Since day 1',
  "@115713 H there! We'd definitely like to work with you on this, how long have you been experiencing this issue? -AA",
  '@115716 What information is incorrect? ^JK',
  '@Ask_Spectrum Would you like me to email you a copy of one since Spectrum is not updating your training?',
  "@115716 Our department is part of the corporate office.  If you're particular area has gone to this format, we were unawa... https://t.co/P7XCmTzPQj",
  '@Ask_Spectrum T

In [15]:
Questions_Fourth = data.Field(sequential = True, tokenize = 'spacy', batch_first =True)#, include_lengths=True)
Answers_Fourth = data.Field(tokenize ='spacy', is_target=True, batch_first =True)#, sequential =False)

In [16]:
fields = [('questions_fourth', Questions_Fourth),('answers_fourth', Answers_Fourth)]

In [17]:
example = [data.Example.fromlist([questions[i],answers[i]], fields) for i in range(len(questions))] 

In [18]:
twitterDataset = data.Dataset(example, fields)

In [19]:
(Train_Dataset_Fourth, Valid_Dataset_Fourth) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [20]:
(len(Train_Dataset_Fourth), len(Valid_Dataset_Fourth))

(2870, 506)

In [21]:
print(vars(Train_Dataset_Fourth.examples[0]))
print(vars(Valid_Dataset_Fourth.examples[0]))

{'questions_fourth': ['@119504', 'We', 'apologize', 'for', 'the', 'misunderstanding', '.', 'You', 'check', 'the', 'status', 'of', 'your', 'order', 'at', 'https://t.co/RDgqxv7W9w', '.', '-', 'KC'], 'answers_fourth': ['@sprintcare', '?', '?', '?', '?', '?', 'https://t.co/1ipFZBAKom']}
{'questions_fourth': ['@118079', 'We', "'d", 'still', 'like', 'to', 'help', '.', 'What', 'does', 'the', 'tracking', 'show', 'in', 'Your', 'Orders', ':', 'https://t.co/PCtircFm5C', '?', 'Who', 'is', 'the', 'carrier', 'here', '?', '^SJ'], 'answers_fourth': ['@AmazonHelp', 'I', 'called', 'customer', 'serv', ',', 'they', 'called', 'usps', 'and', 'the', 'usps', 'rep', 'blamed', 'your', 'routing', '.', 'The', 'second', 'the', 'usps', 'rep', 'was', 'off', 'the', 'phone', 'Amazon', 'blamed', 'usps']}


In [22]:
Questions_Fourth.build_vocab(Train_Dataset_Fourth, min_freq = 2)
Answers_Fourth.build_vocab(Train_Dataset_Fourth, min_freq = 2)

In [23]:
print('Size of input vocab : ', len(Questions_Fourth.vocab))
print('Size of label vocab : ', len(Answers_Fourth.vocab))
print('Top 10 words appreared repeatedly :', list(Questions_Fourth.vocab.freqs.most_common(10)))
print('Labels : ', Answers_Fourth.vocab.stoi)

Size of input vocab :  3453
Size of label vocab :  3338
Top 10 words appreared repeatedly : [('.', 2642), ('the', 1598), ('to', 1547), ('you', 1495), (',', 1340), ('?', 1017), ('I', 991), ('!', 800), ('a', 718), ('and', 686)]
Labels :  defaultdict(<function _default_unk_index at 0x7f92060c0f28>, {'<unk>': 0, '<pad>': 1, '.': 2, 'the': 3, 'to': 4, 'I': 5, ',': 6, 'you': 7, '?': 8, 'a': 9, 'and': 10, '!': 11, 'it': 12, 'for': 13, 'on': 14, 'is': 15, 'your': 16, 'have': 17, 'in': 18, 'this': 19, 'of': 20, 'my': 21, 'we': 22, 'that': 23, 'with': 24, 'can': 25, 'not': 26, 'be': 27, 'are': 28, 'us': 29, 'We': 30, 'do': 31, "n't": 32, 'me': 33, ' ': 34, 'was': 35, '-': 36, "'s": 37, '@AmazonHelp': 38, 'but': 39, 'so': 40, 'will': 41, '@Tesco': 42, 'at': 43, 'Thanks': 44, 'get': 45, 'just': 46, 'an': 47, 'DM': 48, 'n’t': 49, 'out': 50, '#': 51, '...': 52, ':': 53, 'our': 54, 'It': 55, 'Please': 56, 'if': 57, 'from': 58, 'as': 59, 'help': 60, 'no': 61, 'up': 62, '’s': 63, '\n': 64, 'there': 65,

In [24]:
BATCH_SIZE = 128
train_iterator_fourth, valid_iterator_fourth = data.BucketIterator.splits(
                                (Train_Dataset_Fourth, Valid_Dataset_Fourth), 
                                batch_size = BATCH_SIZE,
                                sort_key = lambda x: len(x.questions_fourth),
                                sort_within_batch=True ,
                                device = device)

In [25]:
for i, batch in enumerate(train_iterator_fourth):
  print(type(batch), type(batch.answers_fourth), type(batch.questions_fourth), batch.answers_fourth.shape, batch.questions_fourth.shape)
  break

<class 'torchtext.data.batch.Batch'> <class 'torch.Tensor'> <class 'torch.Tensor'> torch.Size([128, 61]) torch.Size([128, 17])


## Building the Seq2Seq Model

### Encoder

The encoder is similar to the previous one, with the multi-layer LSTM swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

The rest of the encoder should be very familar from the last session, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq5.png?raw=1)

In [26]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

## Decoder

The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder. 

Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq6.png?raw=1)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.

How do these two changes reduce the information compression? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it (don't listen to anyone that says differently). Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [27]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [28]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

# Training the Seq2Seq Model

The rest of this session is very similar to the previous one. 

We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [29]:
INPUT_DIM = len(Questions_Fourth.vocab)
OUTPUT_DIM = len(Answers_Fourth.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [30]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(3453, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(3338, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=3338, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

We print out the number of parameters.

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [31]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,166,346 trainable parameters


We initiaize our optimizer.

In [32]:
optimizer = optim.Adam(model.parameters())

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [33]:
TRG_PAD_IDX = Answers_Fourth.vocab.stoi[Answers_Fourth.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [34]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.questions_fourth
        trg = batch.answers_fourth
        
        src = src.permute(1, 0)
        trg = trg.permute(1, 0)

        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        print(output.shape)
        output = output[1:].view(-1, output_dim)
        trg = trg[1:]
        trg = torch.reshape(trg, (-1,)) #trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [35]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.questions_fourth
            trg = batch.answers_fourth

            src = src.permute(1, 0)
            trg = trg.permute(1, 0)

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            #print(output.shape)
            output = output[1:].view(-1, output_dim)
            trg = trg[1:]#.view(-1)
            trg = torch.reshape(trg, (-1,))

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [36]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [37]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator_fourth, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator_fourth, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model8.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

torch.Size([61, 128, 3338])
torch.Size([42, 128, 3338])
torch.Size([55, 128, 3338])
torch.Size([62, 128, 3338])
torch.Size([57, 128, 3338])
torch.Size([66, 128, 3338])
torch.Size([57, 128, 3338])
torch.Size([68, 128, 3338])
torch.Size([61, 128, 3338])
torch.Size([76, 54, 3338])
torch.Size([61, 128, 3338])
torch.Size([40, 128, 3338])
torch.Size([39, 128, 3338])
torch.Size([58, 128, 3338])
torch.Size([56, 128, 3338])
torch.Size([46, 128, 3338])
torch.Size([55, 128, 3338])
torch.Size([57, 128, 3338])
torch.Size([72, 128, 3338])
torch.Size([64, 128, 3338])
torch.Size([35, 128, 3338])
torch.Size([59, 128, 3338])
torch.Size([60, 128, 3338])
Epoch: 01 | Time: 3m 6s
	Train Loss: 6.904 | Train PPL: 996.314
	 Val. Loss: 5.915 |  Val. PPL: 370.616
torch.Size([76, 54, 3338])
torch.Size([61, 128, 3338])
torch.Size([61, 128, 3338])
torch.Size([68, 128, 3338])
torch.Size([62, 128, 3338])
torch.Size([72, 128, 3338])
torch.Size([58, 128, 3338])
torch.Size([56, 128, 3338])
torch.Size([66, 128, 3338])
to

Finally, we test the model on the test set using these "best" parameters.

In [38]:
#model.load_state_dict(torch.load('tut2-model.pt'))

#test_loss = evaluate(model, test_iterator, criterion)

#print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Just looking at the test loss, we get better performance. This is a pretty good sign that this model architecture is doing something right! Relieving the information compression seems like the way forard, and in the next tutorial we'll expand on this even further with *attention*.

In [ ]:
0#load weights and tokenizer

path='./tut2-model8.pt.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_sentiment(sentence):
    
    categories = {2: "Negative", 1:"Positive", 0:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)
 
    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]